#### <center>======================================================================================</center>
# <center>**Université Paris 1 Panthéon-Sorbonne**</center>
## <center>Projet de Deep Learning</center>
### <center>*Présenté par :* </center>
<center>Berthony Sully</center>

### <center>Au professeur : **Joseph Rynkiewicz** </center>

<center>31 Juillet 2022</center>

#### <center>======================================================================================</center>



# Résumé

Dans ce projet, nous devons réaliser une classification d'images concernant de 3 groupes d'actrices dont deux d'entre eux sont connues pour être des sosies (Natalie Portman et Kiera Nightley). Pour y arriver, nous avons utilisé les CNNs et procédé à certaines opérations en utilisant la framework Pytorch. 

Pour effectuer ce travail, nous disposons d'un dossier d'images conprenant deux sous dossiers:
- `train` qui comprend 429 images 
- `val` qui comprend 168 images
Chacun de ces sous dossiers comprennent eux-mêmes trois sous dossiers dans lesquels sont classés les images en trois classes (`Natalie`, `Kiera`, `Autres`) de format 530x400 pixels.

Ce travail comprend deux grandes parties :
1. La construction d'un réseau avec une configuration experimentale.
2. L'utlisation du le Transfer Learning pour améliorer la classification.

Pour la première partie, nous avons developpé des modèles simples et aussi des modèles assez compliqués qui demandent beaucoup de ressources pour s'entrainer. Nous avons aussi transformé et standardisé les images pour les deux procédures.

Parmi nos modèles définis, celui qui a été le plus performant a été spécifié comme suit :  `config0 = [64,"M", 256, "M", 512, "M"]`. Ce modèle nous a permis d'avoir une accuracy de `55.7%` sur l'ensemble d'entrainement et de `40.5%` sur l'ensemble de validation. Ce qui parait assez pauvre comme résultat mais ce sont les meilleurs résultats qu'on a pu trouver avec toutes ces configurations dont nous avons definies.

En ce qui a trait au tranfer learning, on a pu essayer beaucoup de modèles comme `efficientNet(b0 à b7), ResNet18, ResNet152 et Inception3`. Le résultats dont nous avons trouvé indiquent que le modèle ResNet18 a été le plus adapté pour résoudre ce problème avec `60.7%` d'accuracy sur les données de validation.


# Modèles définis

In [41]:
# Importation des outils
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
import torchvision.transforms as transforms
import os
import numpy as np
from tqdm import tqdm
from torchsummary import summary

In [42]:
# Verifier s'il existe un GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Pour avoir une experience reproductible
torch.manual_seed(0)
np.random.seed(0)

In [43]:
# Parametres de standardisation
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])

# Methode de transformation
train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
])
val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
])


trainset = 'data/train'
validset = 'data/val'

# Création des'ensembles d'entrainement et de validation
train_data = torchvision.datasets.ImageFolder(trainset, transform=train_transform)
val_data = torchvision.datasets.ImageFolder(validset, transform=val_transform)

In [44]:
# Definition des loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=16, shuffle=False)

In [45]:
# verification des caracteristtiques des donnees
[train_data, val_data]

[Dataset ImageFolder
     Number of datapoints: 429
     Root location: data/train
     StandardTransform
 Transform: Compose(
                Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.5 0.5 0.5], std=[0.25 0.25 0.25])
            ),
 Dataset ImageFolder
     Number of datapoints: 168
     Root location: data/val
     StandardTransform
 Transform: Compose(
                Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
                RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.5 0.5 0.5], std=[0.25 0.25 0.25])
            )]

In [47]:
# configuration des modèles
config0 = [64,"M", 256, "M", 512, "M"]

config1 = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M', 256, 128, 64, 32]

config2 = [64, 'M', 128, 'M', 256, 256, 'M', 512,'M', 512, 'M']

config3 = [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

config4 = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512,'M', 512, 512, 512, 'M']

In [48]:
class VGG(nn.Module):
    def __init__(self,architecture):
        super(VGG, self).__init__()
        self.layers = self.make_layers(architecture)
        self.classifier = nn.Linear(512*28*28, 3)

    def forward(self,x):
        x = self.layers(x)
        x = x.view(x.size(0),-1)
        return self.classifier(x)
        
    def make_layers(self,architecture):
        layers = []
        in_channels = 3
        for x in architecture:
            if x=='M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=2, padding=1),
                           nn.BatchNorm2d(x),
                           nn.Softmax()]
                in_channels = x
        return nn.Sequential(*layers)

In [49]:
net = VGG(config0)

net = net.to(device)

In [50]:
summary(net, (3, 224, 224))

c:\Users\bsully\Anaconda3\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 225, 225]             832
       BatchNorm2d-2         [-1, 64, 225, 225]             128
           Softmax-3         [-1, 64, 225, 225]               0
         MaxPool2d-4         [-1, 64, 112, 112]               0
            Conv2d-5        [-1, 256, 113, 113]          65,792
       BatchNorm2d-6        [-1, 256, 113, 113]             512
           Softmax-7        [-1, 256, 113, 113]               0
         MaxPool2d-8          [-1, 256, 56, 56]               0
            Conv2d-9          [-1, 512, 57, 57]         524,800
      BatchNorm2d-10          [-1, 512, 57, 57]           1,024
          Softmax-11          [-1, 512, 57, 57]               0
        MaxPool2d-12          [-1, 512, 28, 28]               0
           Linear-13                    [-1, 3]       1,204,227
Total params: 1,797,315
Trainable param

In [51]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=.001, momentum=0.9)
lr_sc = lr_scheduler.StepLR(optimizer=optimizer, step_size=5)

In [52]:
# Training
def train(epoch,trainloader):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    loop = tqdm(enumerate(trainloader), total=len(trainloader))
    for batch_idx, (inputs, targets) in loop:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        loop.set_description(f"Epoch [{epoch}]")
        loop.set_postfix(acc=correct/total)

In [53]:
def test(epoch,validloader):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        loop = tqdm(enumerate(validloader), total=len(validloader))
        for batch_idx, (inputs, targets) in loop:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            loop.set_postfix(acc=correct/total)
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.t7')
        best_acc = acc

best_acc = 0       

In [54]:
# Testing 
for epoch in range(0, 30):
    train(epoch,train_loader)
    test(epoch,val_loader)
    lr_sc.step()

  0%|          | 0/14 [00:00<?, ?it/s]

Saving..


  0%|          | 0/14 [00:00<?, ?it/s]

Saving..


  0%|          | 0/14 [00:00<?, ?it/s]

Saving..


  0%|          | 0/14 [00:00<?, ?it/s]

Saving..


100%|██████████| 11/11 [00:26<00:00,  2.39s/it, acc=0.345]


# Transfer learning

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [2]:
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

data_dir = 'data/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(class_names)

['keira', 'nathalie', 'others']


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() 
            else:
                model.eval() 
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [67]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001)

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=30)

Epoch 0/29
----------
train Loss: 1.1389 Acc: 0.3520
val Loss: 1.1238 Acc: 0.3869

Epoch 1/29
----------
train Loss: 1.0829 Acc: 0.4009
val Loss: 1.1553 Acc: 0.3869

Epoch 2/29
----------
train Loss: 1.0165 Acc: 0.4732
val Loss: 1.0383 Acc: 0.4762

Epoch 3/29
----------
train Loss: 0.9898 Acc: 0.5175
val Loss: 0.9993 Acc: 0.5000

Epoch 4/29
----------
train Loss: 0.9786 Acc: 0.5315
val Loss: 0.9960 Acc: 0.4702

Epoch 5/29
----------
train Loss: 0.9442 Acc: 0.5478
val Loss: 0.9684 Acc: 0.5357

Epoch 6/29
----------
train Loss: 0.8880 Acc: 0.5828
val Loss: 0.9654 Acc: 0.5357

Epoch 7/29
----------
train Loss: 0.8638 Acc: 0.6294
val Loss: 0.9250 Acc: 0.5417

Epoch 8/29
----------
train Loss: 0.8370 Acc: 0.6573
val Loss: 0.9056 Acc: 0.5774

Epoch 9/29
----------
train Loss: 0.8406 Acc: 0.6480
val Loss: 0.8913 Acc: 0.5595

Epoch 10/29
----------
train Loss: 0.8412 Acc: 0.6503
val Loss: 0.8984 Acc: 0.5655

Epoch 11/29
----------
train Loss: 0.8393 Acc: 0.6107
val Loss: 0.8969 Acc: 0.5476

Ep